In [1]:
import findspark 
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
from pyspark.conf import SparkConf

In [4]:
import re

In [5]:
spark = SparkSession.builder.master("local[4]").appName("rdd_wordcount").getOrCreate()

21/08/22 21:54:43 WARN Utils: Your hostname, zehra-Vostro-5402 resolves to a loopback address: 127.0.1.1; using 192.168.1.105 instead (on interface wlp45s0)
21/08/22 21:54:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/08/22 21:54:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
sc = spark.sparkContext

In [7]:
path = "/home/zehra/forsa.txt"

In [8]:
story_rdd = sc.textFile(path)

In [9]:
story_rdd.count()

95

In [10]:
story_rdd.take(5)

['Forsa - Ömer Seyfettin',
 '',
 'Akdeniz’in, kahramanlık yuvası sonsuz ufuklarına bakan küçük tepe, minimini bir çiçek ormanı gibiydi. İnce uzun dallı badem ağaçlarının alaca gölgeleri sahile inen keçiyoluna düşüyor, ilkbaharın tatlı rüzgârıyla sarhoş olan martılar, çılgın bağrışlarıyla havayı çınlatıyordu. Badem bahçesinin yanı geniş bir bağdı. Beyaz taşlardan yapılmış kısa bir duvarın ötesindeki harabe vadiye kadar iniyordu. Bağın ortasındaki yıkık kulübenin kapısız girişinden bir ihtiyar çıktı. Saçı sakalı bembeyazdı. Kamburunu düzeltmek istiyormuş gibi gerindi. Elleri, ayakları titriyordu. Gök kadar boş, gök kadar sakin duran denize baktı, baktı.',
 '',
 '– Hayırdır inşallah! dedi.']

In [11]:
clean_data = story_rdd.map(lambda row: re.sub(re.compile(r'[^\w\s]'),"",row).lower())
clean_data.take(5)

['forsa  ömer seyfettin',
 '',
 'akdenizin kahramanlık yuvası sonsuz ufuklarına bakan küçük tepe minimini bir çiçek ormanı gibiydi i̇nce uzun dallı badem ağaçlarının alaca gölgeleri sahile inen keçiyoluna düşüyor ilkbaharın tatlı rüzgârıyla sarhoş olan martılar çılgın bağrışlarıyla havayı çınlatıyordu badem bahçesinin yanı geniş bir bağdı beyaz taşlardan yapılmış kısa bir duvarın ötesindeki harabe vadiye kadar iniyordu bağın ortasındaki yıkık kulübenin kapısız girişinden bir ihtiyar çıktı saçı sakalı bembeyazdı kamburunu düzeltmek istiyormuş gibi gerindi elleri ayakları titriyordu gök kadar boş gök kadar sakin duran denize baktı baktı',
 '',
 ' hayırdır inşallah dedi']

In [12]:
words = clean_data.flatMap(lambda row: row.split(" ")).filter(lambda x: x != "")
words.take(5)

['forsa', 'ömer', 'seyfettin', 'akdenizin', 'kahramanlık']

In [13]:
word_count = words.map(lambda word: (word, 1))
word_count.take(5)

[('forsa', 1),
 ('ömer', 1),
 ('seyfettin', 1),
 ('akdenizin', 1),
 ('kahramanlık', 1)]

In [14]:
rbk_wordcount = word_count.reduceByKey(lambda x,y: x+y)
rbk_wordcount.take(15)

[('forsa', 1),
 ('akdenizin', 1),
 ('kahramanlık', 2),
 ('sonsuz', 1),
 ('ufuklarına', 1),
 ('bakan', 1),
 ('gibiydi', 3),
 ('i̇nce', 1),
 ('uzun', 1),
 ('badem', 3),
 ('alaca', 1),
 ('inen', 1),
 ('keçiyoluna', 1),
 ('rüzgârıyla', 1),
 ('sarhoş', 1)]

In [15]:
rbk_wordcount = rbk_wordcount.map(lambda x: (x[1], x[0]))
rbk_wordcount.take(15)

[(1, 'forsa'),
 (1, 'akdenizin'),
 (2, 'kahramanlık'),
 (1, 'sonsuz'),
 (1, 'ufuklarına'),
 (1, 'bakan'),
 (3, 'gibiydi'),
 (1, 'i̇nce'),
 (1, 'uzun'),
 (3, 'badem'),
 (1, 'alaca'),
 (1, 'inen'),
 (1, 'keçiyoluna'),
 (1, 'rüzgârıyla'),
 (1, 'sarhoş')]

In [16]:
rbk_wordcount.sortByKey(False).take(20)

[(36, 'bir'),
 (8, 'kırk'),
 (8, 'yıl'),
 (6, 'onu'),
 (6, 'diye'),
 (6, 'bu'),
 (6, 'doğru'),
 (5, 'baktı'),
 (5, 'tutsak'),
 (5, 'yirmi'),
 (5, 'evet'),
 (5, 'i̇htiyar'),
 (5, 'ben'),
 (5, 'dedi'),
 (5, 'türk'),
 (5, 'onun'),
 (5, 'büyük'),
 (4, 'daha'),
 (4, 'rüya'),
 (4, 'gibi')]